In [1]:
import pandas as pd

In [7]:
df_session = pd.read_csv('/media/workspace/triplet_session/output/yoochoose/dataset/train_0.01_test=random_42_none_PrepareSessionInteractionsDataFrames_available_arms____None_6e929f94bd.csv')
df_session.shape

(60928, 12)

In [8]:
df_session.head()

,SessionID,ItemID_A,pos_A,ItemID_B,pos_B,relative_pos,total_ocr,prob,sub_a_b,ItemID,available_arms,visit
0,221307,214835390,0,214835368,1,1,77,0.051948,"[214835368, 214600922, 214613685, 214835385, 2...",214835390,"[214510700, 214531612, 214536386, 214537745, 2...",1.0
1,18141,214835816,2,214819438,0,2,75,0.040000,"[214581504, 214836232, 214836114, 214836118, 2...",214835816,"[214532000, 214534667, 214538430, 214539399, 2...",1.0
2,11229,214839755,5,214835760,4,1,569,0.005272,"[214587650, 214835717, 214820231, 214833800, 2...",214839755,"[214534217, 214537751, 214541060, 214550390, 2...",1.0
3,354888,214826615,0,214573288,1,1,2556,0.003130,"[214711297, 214791682, 214757379, 214553604, 2...",214826615,"[214512673, 214530789, 214536386, 214539403, 2...",1.0
4,307293,214820300,3,214591725,5,2,61,0.065574,"[214718405, 214706280, 214820300, 214591725, 2...",214820300,"[214513063, 214514293, 214518817, 214531355, 2...",1.0


In [9]:
df_train = pd.read_csv('/media/workspace/triplet_session/output/yoochoose/dataset/train_0.01_test=random_42_none_SessionInteractionDataFrame_AvailableItems____None_7a340ca010.csv')
df_train.shape

(812394, 7)

In [10]:
df_train.head()

,SessionID,Timestamp,ItemID,Category,ItemIDHistory,AvailableItems,visit
0,171168,2014-04-01 00:00:32.610,214820231,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 214594678]","[214834980, 214658280, 214552492, 214832383, 2...",1
1,389654,2014-04-01 00:00:48.736,214587557,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 214826705]","[214507408, 214631735, 214730518, 214600479, 2...",1
2,375257,2014-04-01 00:01:03.866,214582935,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 214644307]","[214674857, 214559019, 214555277, 214694756, 2...",1
3,263073,2014-04-01 00:01:07.619,214716982,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 214716982]","[214544936, 214691248, 214836997, 214692718, 2...",1
4,345618,2014-04-01 00:01:15.896,214705119,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 214687685]","[214552162, 214669867, 214708018, 214586910, 2...",1


In [14]:
df_item_s = df_session[['ItemID']].drop_duplicates()
df_item_s.shape

(7004, 1)

In [12]:
df_item_t = df_train[['ItemID']].drop_duplicates()
df_item_t.shape

(19945, 1)